# Script pour collecter des tweets

### Imports

In [1]:
import os
import json
from time import time
from datetime import datetime
from twython import TwythonStreamer

### Paramètres
On configure le streamer. 
- `max_tweets`, `max_time`: on choisit un nombre maximal de tweets et/ou un temps maximal de crawling. Si les 2 sont précisés on s'arrête que l'un est atteint.
- `print_tweets` : pour afficher les tweets au fur et à mesure qu'ils arrivent.

In [2]:
max_tweets = 3
max_time = None
print_tweets = True

Les tags que l'on cherche.

In [3]:
tags = [ "@katyperry", "@justinbieber", "@BarackObama", "@rihanna", "@taylorswift13", "@ladygaga" ]

### Fonction de stream

In [4]:
def streamer(Tags, Max_tweets, Max_time, Print_tweets):
    
        
    # les key et token pour l'api
    APP_KEY = "AV4KujYNS1y6HdmQ4zXxHw2Tx"
    APP_SECRET = "xd2FuFaS5zE1ON407C3U5X0kdEtRJIjH9OMDmgyMmxQCZMFlli"
    OAUTH_TOKEN = "1058011325316636672-HkpzhyiOHdJkuwi3FD9QrkMFbuHuVt"
    OAUTH_TOKEN_SECRET = "iiD75H9ERtkATnd74pVRBfM7TI189BSmDasYzN2uUO123"
    
    
    # la liste des tweets qu'on va enregistrer
    tweets = list()
    
    
    # streamer 
    class MyStreamer(TwythonStreamer):
        def __init__(self, app_key, app_secret, oauth_token, oauth_token_secret, max_tweets=10, max_time=None, timeout=300, retry_count=None, retry_in=10, client_args=None, handlers=None, chunk_size=1):
            TwythonStreamer.__init__(self, app_key, app_secret, oauth_token, oauth_token_secret, timeout=300, retry_count=None, retry_in=10, client_args=None, handlers=None, chunk_size=1)
            self.__startingTime__ = time()
            self.__maxTime__ = max_time
            self.__nbTweets__ = 0
            self.__maxTweets__ = max_tweets
        def on_success(self, data):
            if 'text' in data:
                self.__nbTweets__ += 1
                if Print_tweets:
                    print(data['text'])
                    print(data['created_at'])
                    print(str(data['id'])+'\n')
                tweets.append(data)
            if self.__maxTime__ != None:
                if time()-self.__startingTime__ >= self.__maxTime__:
                    self.disconnect()
            if self.__maxTweets__ != None:
                if self.__nbTweets__ == self.__maxTweets__:
                    self.disconnect()
        def on_error(self, status_code, data):
            print(status_code)
            self.disconnect()
    
        
    # on stream
    stream = MyStreamer(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET, max_tweets=Max_tweets, max_time=Max_time)
    stream.statuses.filter(track=tags, language='fr', tweet_mode='extended')
    
    
    
    # on renvoie les tweets collectés
    return tweets

### Exécution

In [5]:
start = datetime.now().strftime("%d/%m/%Y-%H:%M")
Tweets = streamer(tags, max_tweets, max_time, print_tweets)
end = datetime.now().strftime("%d/%m/%Y-%H:%M")

RT @NRJhitmusiconly: Allez, dites-nous ➡️ quel hit de #JustinBieber êtes-vous ? 😍
👉 Pour son anniversaire, on passe en mode @justinbieber s…
Fri Mar 01 16:06:45 +0000 2019
1101514119088431109

RT @jdb_cagla: Vote Vote Vote 
#iHeartAwards #beliebers #BestFanArmy @justinbieber https://t.co/rZgPYaZiYj
Fri Mar 01 16:07:03 +0000 2019
1101514192425807872

RT @NRJhitmusiconly: Allez, dites-nous ➡️ quel hit de #JustinBieber êtes-vous ? 😍
👉 Pour son anniversaire, on passe en mode @justinbieber s…
Fri Mar 01 16:07:13 +0000 2019
1101514235614420993



### Enregistrement
Les tweets sont enregistrés sous la forme `crawl#.json` où # est le numéro du crawl. On enregistre des infos dans un fichier central `infos.txt`.

In [6]:
# le numero d'enregistrement
crawls = [ file for file in os.listdir('collected/') if file.startswith('crawl') ]
if len(crawls) == 0:
    num = 1
else:
    num = sorted([int(c[5:-5]) for c in crawls])[-1] + 1

# on enregistre les tweets
filename = 'collected/crawl{}.json'.format(num)
json.dump(Tweets, open(filename, 'w'))

# les infos
with open("collected/infos.txt", 'a') as infos:
    infos.write('{} : from {} to {}, with {} tweets\n'.format(num, start, end, len(Tweets)))
infos.close()

In [7]:
data = json.load(open('crawl1.json','r'))

In [8]:
data

[{'created_at': 'Fri Mar 01 13:58:05 +0000 2019',
  'id': 1101481737526362112,
  'id_str': '1101481737526362112',
  'text': "RT @MediasUpr: 🔴 F.#Asselineau une semaine avant le 1er rassemblement des #GiletsJaunes (10/11/18) : \n\n« On est un peu dans la situation d'…",
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'truncated': False,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 3312195466,
   'id_str': '3312195466',
   'name': 'ian',
   'screen_name': 'fulei777',
   'location': None,
   'url': None,
   'description': None,
   'translator_type': 'none',
   'protected': False,
   'verified': False,
   'followers_count': 140,
   'friends_count': 298,
   'listed_count': 17,
   'favourites_count': 21756,
   'statuses_count': 22610,
   'created_at': 'Sun Jun 07 17:39:36 +0000 2015',
   'utc_offse